# 10章 データの集約とグループ演算

データをカテゴライズして、各グループに関数を適用することは、集約や変換と呼ばれ、データ分析のワークフローの中で最も重要な部分である。

リレーショナルデータベースとSQLが一般的なのは、データの連結やフィルタリング、変換、集約が容易であるからであるが、SQLのようなクエリ言語はグループ演算で実現できることにある程度の制約がある。Pythonのpandasの表現力を使えば、より複雑なグループ演算を行うことができる。ここでは次のことを学ぶ：

- pandasのオブジェクトを1つあるいは複数のキーで分割する方法
- グループの要約統計量の計算方法
- グループ内の変換やその他のデータ操作
- ピボットテーブルとクロス集計の計算
- 分位点分析やその他のデータから生成されたグループに関する分析

## 10.1 GroupByの仕組み

R言語で多くの有名なパッケージを作っているHadley Wickhamによってグループ演算のプロセスを説明するために作られた言葉が**分離**-**適用**-**結合**（split-apply-combine）である。プロセスの最初の段階では、pandasオブジェクトに格納されているデータが1つ以上のキーによって分離され、次に各グループに関数が適用され、新しい値が生成される。最後に、これらの関数を適用した結果が結合されて結果を戻すオブジェクトに格納される。

グループ化に使用するキーはさまざまな形式をとることができ、またすべて同じ型である必要もない。値のリストや配列、データフレームの列名を示す値、辞書形式あるいはシリーズ形式、軸のインデックスあるいは各インデックスのラベルに対して呼び出される関数などである。

In [6]:
import numpy as np
import pandas as pd

In [7]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'], 
                   'key2': ['one', 'two', 'one', 'two', 'one'], 
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.084314,0.817057
1,a,two,-1.736081,-0.781876
2,b,one,0.397605,-1.633213
3,b,two,-0.469188,-1.131233
4,a,one,-1.158020,0.821505


``key1``のラベルでグループ化して、``data1``列の平均値を計算するには、さまざまな方法があるが、その1つが``data1``にアクセスして、``groupby``にグループする列として``key1``を指定して呼び出す方法である。

In [8]:
grouped = df['data1'].groupby(df['key1'])
grouped

グループ化された変数は、この時点では**GroupBy**オブジェクトになっている。このオブジェクトには実際に計算した結果は含まれておらず、グループキー``df['key1']``に関する中間データが含まれている状態である。例えば各グループの平均値を計算するために、GroupByの``mean``メソッドを呼ぶことができる。

In [9]:
grouped.mean()

key1
a   -0.936596
b   -0.035791
Name: data1, dtype: float64

グループキーに``key1``だけを指定する代わりに、複数の配列をリストとして渡すと、少し異なる結果が得られる。次の例ではデータを2つのキーを使ってグループ化しているため、結果のシリーズは、キーで観測された独立な組み合わせが含まれる階層的なインデックスになっている。

In [11]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.536853
      two    -1.736081
b     one     0.397605
      two    -0.469188
Name: data1, dtype: float64

In [12]:
means.unstack()

key2,one,two
key1,,
a,-0.536853,-1.736081
b,0.397605,-0.469188


上の例ではグループキーはすべてシリーズ型だったが、正しい長さの配列をグループキーに使うこともできる。

In [13]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -1.736081
            2006    0.397605
Ohio        2005   -0.192437
            2006   -1.158020
Name: data1, dtype: float64

グループ化に使いたい情報がデータフレームの中のデータとして存在するならば、グループキーとしてそのデータが含まれる列名を渡すことができる。次の2つの例のうち最初の例では結果に``key2``列が含まれていないことに注意されたい。これは``df['key2']``が数値ではない列（邪魔な列と呼ぶ）であるため結果から除外されているからである。デフォルトではすべての数値列が集約されるが、後述の方法でフィルタリングすることもできる。

In [14]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.936596,0.285562
b,-0.035791,-1.382223


In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.536853  0.819281
     two  -1.736081 -0.781876
b    one   0.397605 -1.633213
     two  -0.469188 -1.131233

``groupby``の使用目的によらず、便利なGroupByオブジェクトのメソッドに``size``メソッドがある。これは、各グループのサイズ情報を持つシリーズを戻す。

In [16]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 グループをまたいだ繰り返し

GroupByオブジェクトは繰り返しをサポートし、繰り返しの中では、グループの名前(``name``)とその名前に対応するデータ（``group``）の2つを含むタプルで構成されるシーケンスを生成する。

In [17]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.084314  0.817057
1    a  two -1.736081 -0.781876
4    a  one -1.158020  0.821505
b
  key1 key2     data1     data2
2    b  one  0.397605 -1.633213
3    b  two -0.469188 -1.131233


複数キーを扱うケースでは、繰り返しの中で使用されるタプルに含まれる最初の要素は、キーの値のタプルになっている。

In [18]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.084314  0.817057
4    a  one -1.158020  0.821505
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.736081 -0.781876
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.397605 -1.633213
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.469188 -1.131233


取り出したいデータはどのようなものであっても選択することができる。便利な方法の1つに、データを辞書形式に変換する処理を1行で書くやり方がある（キー列の各カテゴリ値をキーとした辞書の作り方）。

In [23]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.397605,-1.633213
3,b,two,-0.469188,-1.131233


デフォルトの``groupby``のグループは``axis=0``に設定されるが、グループは別の軸（``axis``）に設定することができる。例えば、例に使っている``df``の例を``dtype``を使って次のようにグループ化することができる（各列ごとに決まる属性をキー列に用いた例）。

In [24]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [27]:
grouped = df.groupby(df.dtypes, axis=1)
for name, group in grouped:
    print(name)
    print(group)

float64
      data1     data2
0  0.084314  0.817057
1 -1.736081 -0.781876
2  0.397605 -1.633213
3 -0.469188 -1.131233
4 -1.158020  0.821505
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 列や列の集合の選択

GroupByオブジェクトに対して、インデックス参照するということは、集約する列を選択するのと同じ効果がある。

In [29]:
df.groupby('key1')['data1']
df.groupby('key1')['data2']

これは次のコードのシンタックスシュガーである。

In [31]:
df['data1'].groupby(df['key1'])
df['data2'].groupby(df['key1'])

特に大きなデータセットの場合、わずかな列だけで集約する方が望ましい場合がある。例えば先ほどのデータセットに対し、``data2``列だけの平均を計算し結果をデータフレームで得たければ次のように書ける。

In [34]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.819281
     two  -0.781876
b    one  -1.633213
     two  -1.131233

結果をシリーズで得たければ次のように書ける。**リストや配列を与えたときにはグループ化されたデータフレームになり、スカラーとして単独の列名を与えた場合には、グループ化したシリーズになる。**

In [36]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [37]:
df.groupby(['key1', 'key2'])[['data2']]

In [38]:
s_grouped.mean()

key1  key2
a     one     0.819281
      two    -0.781876
b     one    -1.633213
      two    -1.131233
Name: data2, dtype: float64

### 10.1.3 ディクショナリやシリーズのグループ化

グループ化の情報は、配列以外の形式の場合もある。

In [40]:
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.725683,0.262963,0.266812,0.043223,-0.385729
Steve,1.609434,-2.605582,0.764385,-0.673775,-0.127208
Wes,-0.723916,NaN,NaN,1.499292,1.269094
Jim,1.406395,-0.329899,0.516796,-0.816906,-0.107068
Travis,-0.587580,-0.550687,-0.726707,-0.309923,-2.452024


ここでは、どの列をグループ化したいのかを示した（次のような）マッピング情報があるとする（``'f'``もキーに加えているのは使用しないグループ化のキーがあってもOKなのが分かるようにするためである）。

In [42]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [43]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.310035,0.602917
Steve,0.090611,-1.123357
Wes,1.499292,0.545179
Jim,-0.300110,0.969429
Travis,-1.036630,-3.590291


In [44]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [45]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 関数を使ったグループ化 

辞書やシリーズを使ったグループ化方式と比べて、Pythonの関数を使う方法はより汎用的である。グループキーとして渡される関数は、インデックスの値ごとに呼び出され、各戻り値が各グループ名として用いられる。

In [46]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.408163,-0.066936,0.783608,0.725609,0.776298
5,1.609434,-2.605582,0.764385,-0.673775,-0.127208
6,-0.587580,-0.550687,-0.726707,-0.309923,-2.452024


In [47]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.723916  0.262963  0.266812  0.043223 -0.385729
  two  1.406395 -0.329899  0.516796 -0.816906 -0.107068
5 one  1.609434 -2.605582  0.764385 -0.673775 -0.127208
6 two -0.587580 -0.550687 -0.726707 -0.309923 -2.452024

### 10.1.5 インデックス階層によるグループ化

階層を持つインデックスを使う際に決定的に便利なのは、軸のインデックスの階層を使って集約ができる機能である。

In [50]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], 
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.899542 -0.510287  0.179097  0.551857 -0.143151
1      0.101460 -0.564802 -1.460918 -0.284682 -1.838031
2      2.257158  1.145731 -0.117686  0.653285 -0.739900
3      0.353013 -1.543169 -0.373914 -0.531167  1.579984

このデータフレームを階層ごとに集約するには、階層の番号や``level``キーワードを使って階層を指定する。

In [51]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 データの集約

集約とは、何らかのデータ変形を行って配列からスカラー値を生成することを指す。平均や最大・最小をとる操作などは集約であり、GroupByメソッドとして最適化された実装がすでに存在する。自分自身で考えた集約処理を使うこともできる。また、例えば``quantile``メソッドはGroupByオブジェクトで実装されたものではないが、シリーズのメソッドであるため、ここで使うことができる。内部的にはGroupByはシリーズをうまくスライスし、スライスした各ピースに対して``piece.quantile(0.9)``を呼び出している。

In [52]:
df

,key1,key2,data1,data2
0,a,one,0.084314,0.817057
1,a,two,-1.736081,-0.781876
2,b,one,0.397605,-1.633213
3,b,two,-0.469188,-1.131233
4,a,one,-1.158020,0.821505


In [54]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a   -0.164153
b    0.310926
Name: data1, dtype: float64

自分自身で定義した集約関数を使うには、配列を集約する関数を``aggregate``あるいは``agg``メソッドに渡す。

In [55]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [56]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.820395,1.603381
b,0.866793,0.501980


``describe``のようなメソッドも同様に機能することもが、これらのメソッドは厳密には集約ではない。

In [57]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.936596  0.930178 -1.736081 -1.447051 -1.158020 -0.536853   
b      2.0 -0.035791  0.612915 -0.469188 -0.252490 -0.035791  0.180907   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.084314   3.0  0.285562  0.924431 -0.781876  0.017590  0.817057   
b     0.397605   2.0 -1.382223  0.354953 -1.633213 -1.507718 -1.382223   

                          
           75%       max  
key1                      
a     0.819281  0.821505  
b    -1.256728 -1.131233

**独自に作成した集約関数はGroupByメソッド（最大最小をとるなどの最適化済みの関数）に比べて一般的に遅いことに注意されたい。**これは、処理の中間でグループ化したデータの破片を生成するときに、関数の呼び出しやデータの整形に追加でオーバーヘッドがかかってしまうからである。

### 10.2.1 列に複数の関数を適用する

これまではシリーズやデータフレームのすべての列に同じ関数を適用してきた。しかし、列ごとに異なる関数を使って集約したり、複数の関数を同時に使って集約することも可能である。ここでは``tip``データを用いる。

In [3]:
tips = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv')

In [61]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head(6)

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


GroupByメソッドは関数の名前を文字列として渡すことができる。

In [63]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

1つの関数を指定するのではなく、関数や関数の名前のリストを指定した場合、戻り値として関数名と同じ列名を持つデータフレームを得ることができる。

In [65]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

列名を変更したい場合は、(名前，関数)という形式のタプルをリストで渡せば、各タプルの最初の要素がデータフレームの列名として使われる。これは例えばlambda関数を用いたときなどは``'<lambda>'``という名前が表示されるため有用である（関数の``__name__``属性で確認できる）。

In [66]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

データフレームに対しては、さらに別のオプションも使える。関数のリストを指定すれば、すべての列に関数を適用したり、列ごとに異なる関数を適用したりすることができる。

In [68]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

先ほどと同じように、名前を指定したタプルのリストを渡すこともできる。

In [70]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

さらに、複数の列に対して、それぞれ異なる関数を適用したい場合を考える。この場合は、列名と適用したい関数名をマッピングした辞書を``agg``に渡す。

In [71]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [72]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 10.2.2 集約されたデータをインデックスなしで戻す

ここまでに紹介した例では、集約されたデータはインデックス付けされていて、場合によっては階層的インデックスを持ち、一意なグループキーを基に構成されたものだった。しかし、この操作は無効化することができる。このオプションのときは不必要な計算が回避される。

In [73]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 applyメソッド：一般的な分離-適用-結合の方法

最も一般的な目的をもつGroupBYオブジェクトは``apply``である。

まずは先ほどのチップデータで``tip_pct``値のグループ別上位5件を選択したい場合を考える。まずは特定の列の上位の値を持つ行を選択する関数を書く。

In [76]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [77]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


ここで、``smoker``列でグループ分けし、``apply``を使ってこの``top``関数を適用すると、次のような結果になる：

In [78]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

ここで起きたことは、``top``関数がデータフレームの各行グループに対して呼ばれ、結果が``pandas.concat``で結合され、各グループに名前が付いた。したがって、その結果は階層的なインデックスを持ち、内側の階層にもともとのデータフレームのインデックス値を持っている。

引数やキーワードが必要な関数を``apply``に渡す場合でも、その関数の後にそれらの引数を指定することができる。

In [79]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

**``apply``に渡す関数は``pandas``が提供するものではなく自分で実装する必要があるが、それはさらに``pandas``オブジェクトがスカラー値を戻す必要がある**ことに注意されたい。

ここで、以前の``describe``メソッドを振り返る。

In [83]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


GroupByオブジェクトの内部では、``describe``のような関数を呼び出したときには、実際には次のようなコードのショートカットとして機能する。

In [84]:
# f = lambda x: x.describe()
# grouped.apply(f)

### 10.3.1 グループキーの抑制

先ほどの例では、結果のオブジェクトは階層的なインデックスを持ち、各グループキーごとに、もともとのオブジェクトのインデックスを持つような形式になっていた。この仕様は``group_keys=False``を``groupby``に渡すことによって無効にすることができる（最も左の列（グループキー列）がなくなるだけ）。

In [88]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 10.3.2 分位点とビン分析

pandasはいくつかのツールを持っている。例として``cut``や``qcut``が挙げられる。``cut``はデータを同じ長さの便に分割して入れる動作をし（ゆえに各ビン内のサンプルサイズはバラバラ）、``qcut``はサンプルデータの分位点でデータを分割する。

In [93]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-3.189, -1.655]
1       (-0.128, 1.4]
2       (-0.128, 1.4]
3       (-0.128, 1.4]
4    (-3.189, -1.655]
5    (-3.189, -1.655]
6       (-0.128, 1.4]
7    (-3.189, -1.655]
8    (-1.655, -0.128]
9    (-3.189, -1.655]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.189, -1.655] < (-1.655, -0.128] < (-0.128, 1.4] < (1.4, 2.928]]

``cut``メソッドで戻される``Categorical``オブジェクトは、そのまま``groupby``に渡すことができる。そのため、``data2``列の統計量を次のようにして計算することができる。

In [94]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

In [95]:
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.189, -1.655]",60.0,1.635468,-0.067869,-2.700925
"(-1.655, -0.128]",386.0,2.617892,0.008395,-2.941882
"(-0.128, 1.4]",471.0,2.932740,-0.061134,-3.056795
"(1.4, 2.928]",83.0,1.722305,-0.037009,-2.186658


### 10.3.3 例：グループ固有の値で欠測値を埋める

欠測値をクリーニングしたい場合には、``dropna``を使って欠測値を削除する場合もあるが、固定値やデータから導出した値でnull値を穴埋めしたい場合もある。``fillna``はこれを実現するのに適切なツールであり、次にあげる例では欠測値を平均値で埋めている。

In [96]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.317589
2         NaN
3    0.108633
4         NaN
5   -0.769095
dtype: float64

In [97]:
s.fillna(s.mean())

0   -0.114291
1    0.317589
2   -0.114291
3    0.108633
4   -0.114291
5   -0.769095
dtype: float64

欠測値を埋める値をグループによって変えたい場合を考える。これを行うためには、データをグループ分けして、各グループに対して``fillna``を使う関数を``apply``に渡すことである。アメリカの州を東西で分割したサンプルデータの例でこれを試してみる。

In [99]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.324556
New York     -0.456488
Vermont            NaN
Florida      -0.924102
Oregon       -0.381126
Nevada             NaN
California    0.215506
Idaho              NaN
dtype: float64

欠測値をグループの平均値で埋めるには、次のようにする。

In [100]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.324556
New York     -0.456488
Vermont      -0.352011
Florida      -0.924102
Oregon       -0.381126
Nevada       -0.082810
California    0.215506
Idaho        -0.082810
dtype: float64

場合によっては、グループによって変化する、あらかじめコードの中に定義された値を使いたいかもしれない。グループは``name``属性を持っているため、それを使う。

In [101]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.324556
New York     -0.456488
Vermont       0.500000
Florida      -0.924102
Oregon       -0.381126
Nevada       -1.000000
California    0.215506
Idaho        -1.000000
dtype: float64

### 10.3.4 例：ランダムサンプリングと順列

トランプのデータフレームを作成し、スートごとにいくつかずつを抽出する処理を``groupby``を使って行う。

In [102]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [103]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

カード名の最後の文字（インデックスの各要素の最後の文字）がスートを表しているため、2つのランダムなカードをそれぞれのスートから取り出したい場合は、これを利用してグループ化し、``apply``を使う。

In [104]:
def draw(deck, n=5):
    return deck.sample(n)

get_suit = lambda card: card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  9C      9
   4C      4
D  AD      1
   3D      3
H  10H    10
   7H      7
S  AS      1
   5S      5
dtype: int64

### 10.3.5 例：グループの加重平均と相関

``groupby``の分離、適用、結合のパラダイムのもとでは、グループの加重平均値のような、データフレームの複数の列間の操作や2つのシリーズの操作も可能である。

In [106]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.238917,0.846347
1,a,0.773553,0.171205
2,a,-0.472189,0.406751
3,a,1.112202,0.439154
4,b,0.581769,0.353130
5,b,-0.948455,0.636968
6,b,-1.275630,0.754330
7,b,0.016289,0.835520


In [107]:
grouped = df.groupby('category')

get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

grouped.apply(get_wavg)

category
a    0.121598
b   -0.522233
dtype: float64

別の例として、Yahoo!Finaceから取得した金融データを考えてみる。

In [4]:
close_px = pd.read_csv('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/stock_px_2.csv', parse_dates=True, index_col=0)

In [108]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [109]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


日時の利益（パーセント変化から算出）とSPXとの年次の相関を調べるために、まず特定の列と``SPX``との相関を計算する関数をつくる。

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX'])

次に``pct_change``を使って``close_px``のパーセント変化を計算する。

In [ ]:
rets = close_px.pct_change().dropna()

最後に、これらの年次のパーセント変化をグループ化する。年次の「年」は各``datetime``ラベル（ラベル：インデックス値のこと）の``year``属性を戻す1行のラムダ関数を使って、各行から抽出できる。

In [ ]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

列間の相関を計算することもできる。AppleとMicrosoftの年次の相関を計算できる。

In [ ]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

## 10.4 ピボットテーブルとクロス集計

ピボットテーブルは、表計算プログラムやその他のデータ分析ソフトウェアでよくみられるデータの要約ツールである。ピボットテーブルでは、データは1つ以上のキーによってテーブル形式のデータを集約し、特定のグループキーを行に対して、また別のグループキーを列に対して整理し、全体として長方形の形にデータを整形する。``pandas``におけるピボットテーブルは、この章で紹介した``groupby``の機能と階層型のインデックスを操作する再形成機能の組み合わせで実現されている。データフレームには``pivot_table``というメソッドがある。また、トップレベルの``pandas.pivot_table``関数もある。

ここでは、チップのデータセットに戻って、日付（``day``）と喫煙の有無（``smoker``）によって整理されたグループの平均値（``pivot_table``の集約のデフォルトのタイプ）を計算する場合を考える。

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

上は``groupby``を直接使ってもできることであった。次は``tip_pct``と``size``だけを集約し、``time``ごとにグループ化してみる。``smoker``をテーブルの列に配置し、``day``を行に配置する。

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

このテーブルに小計の情報（ALLの列）を追加するには、``margins=True``を指定する。この指定をすると``All``という行と列が追加され、その行や列にあるデータの集計値が``All``の値として表示される。次では、``All``列の値は喫煙者と非喫煙者の区別なく計算された平均値になっている。``All``行の値は、行が持つ2つの階層のすべてを同じグループとしてまとめ、その平均値を求めるている。

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

異なる集約関数を使うには、``aggfunc``に関数を渡す。例えば、``count``や``len``関数は、グループの細部をクロス集計する機能を提供する。クロス集計はピボットテーブルの特殊ケースであり、グループの出現頻度を計算するものである。

In [2]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

NameError: name 'tips' is not defined

いくつかの組み合わせが空白（もしくは欠測値）だった場合、次のように``fill_value``を指定して穴埋めをしたくなるかもしれない。

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

クロス集計はピボットテーブルの特殊なケースであり、グループの出現頻度を計算するものである。 

In [9]:
data = pd.DataFrame({'Sample': list(range(0, 10)), 
                     'Nationality': ['USA', 'Japan', 'USA', 'Japan', 'Japan', 'Japan', 'USA', 'USA', 'Japan', 'USA'],
                     'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']})
data

,Sample,Nationality,Handedness
0,0,USA,Right-handed
1,1,Japan,Left-handed
2,2,USA,Right-handed
3,3,Japan,Right-handed
4,4,Japan,Left-handed
5,5,Japan,Right-handed
6,6,USA,Right-handed
7,7,USA,Left-handed
8,8,Japan,Right-handed
9,9,USA,Right-handed


調査分析の中で、このデータを国籍と聞き手で集計したい場合、``pivot_table``を使うこともできるが、``pandas.crosstab``関数を使うと便利である。

In [10]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


``crossrab``の最初の2つの引数は、配列、シリーズ、配列のリストを使うことができる。チップのデータの場合、次のようになる。

In [11]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

NameError: name 'tips' is not defined

## Reference

- Wes McKinney. Pythonによるデータ分析入門. 第2版, 株式会社オライリー・ジャパン, 2018, 571p.